# Retrieval Augmented Generation using ChromaDB and GPT

## Overview

This code snippet explores the implementation of a conversational chatbot using various APIs and the `langchain` Python library. Specifically, it initiates API interactions with `Cohere` and `OpenAI`, utilizes web scraping for document loading, applies token-based text splitting, and leverages embedding models for semantic understanding. Additionally, it applies a memory buffer to retain chat history and structures conversational chains for query and answer generation. 

### Setting Up API Keys

```python
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI API Key:")
```

The above code securely retrieves API keys for Cohere and OpenAI, prompting the user to input them without displaying them on-screen.

### Loading and Pre-processing Data

```python
urls = [...]
loader = WebBaseLoader(urls)
data = loader.load()
```

A predefined set of URLs is loaded and parsed using `WebBaseLoader`, which essentially fetches documents/web pages for further interaction.

### Embeddings and Document Representation

```python
embeddings = CohereEmbeddings(model='...', cohere_api_key=..("COHERE_API_KEY"))
db = Chroma.from_documents(docs, embeddings)
```
This segment prepares document embeddings using the `CohereEmbeddings` and stores them in a format suitable for quick retrieval and similarity search using `Chroma`.

### Retrievers and Compressors

```python
from langchain.retrievers import ContextualCompressionRetriever, VectorStoreRetriever
from langchain.retrievers.document_compressors import CohereRerank

#... [various retriever and compressor setup]
```
Retrievers and compressors manage the extraction and summarization of information from the loaded documents.

### Conversational Interface

```python
memory = ConversationTokenBufferMemory(llm=llm, memory_key="chat_history", return_messages=True, input_key='question', max_token_limit=1000)
```
The conversation’s state is managed through memory objects. This setup helps retain and reference past interactions to manage coherent dialogue.

### Question and Answer Generation

```python
#... [prompt template set up]

question_generator = LLMChain(llm=llm, prompt=chat_prompt_for_ques, verbose=True)
answer_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True, prompt=chat_prompt)
```
Using predefined prompt templates, the system generates questions and answers. These questions derive from user inputs and historical chat data, while answers reference and cite embedded documents.

### Conversational Chain and User Interaction

```python
chain = ConversationalRetrievalChain(
            retriever=reranker,
            question_generator=question_generator,
            combine_docs_chain=answer_chain,
            verbose=True,
            memory=memory,
            rephrase_question=False
)

query = "What is the Scheduling Policies for HPC cluster?"
result = chain({"question": query})

print("Question from user : " , query ,"\n")
print("Reply from ChatBot : " , result['answer'])
```
This conversational chain consolidates the previously defined components to facilitate user interaction. It retrieves and generates responses dynamically, considering prior conversational history and the loaded document base. 


In [1]:
#!pip install pinecone-client

In [2]:
#!pip install langchain

In [3]:
#!pip install tiktoken

In [4]:
#!pip install cohere

In [5]:
#!pip install openai

In [6]:
#!pip install chromadb

In [7]:
#import pinecone


In [8]:
#pinecone.init(api_key="b360318b-4fc8-4580-bf6c-d88959179985",
#              environment="us-west1-gcp-free")


In [9]:
#pinecone.whoami()

In [10]:
#pinecone.list_indexes()

In [11]:
#pinecone.list_indexes()

In [12]:
import os
import getpass

os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key: ········································


In [13]:
'''
CohereAPIError: You are using a Trial key, which is limited to 2 API calls / minute.
You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at
'https://dashboard.cohere.ai/api-keys'. 
Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions
'''

"\nCohereAPIError: You are using a Trial key, which is limited to 2 API calls / minute.\nYou can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at\n'https://dashboard.cohere.ai/api-keys'. \nContact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions\n"

In [14]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI API Key:")

OPENAI API Key: ···················································


In [15]:
import langchain

In [16]:
from langchain.document_loaders import WebBaseLoader


urls = ["https://rc-docs.northeastern.edu/en/latest/welcome/index.html",
"https://rc-docs.northeastern.edu/en/latest/welcome/welcome.html",
"https://rc-docs.northeastern.edu/en/latest/welcome/services.html",
"https://rc-docs.northeastern.edu/en/latest/welcome/gettinghelp.html",
"https://rc-docs.northeastern.edu/en/latest/welcome/introtocluster.html",
"https://rc-docs.northeastern.edu/en/latest/welcome/casestudiesandtestimonials.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/index.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/get_access.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/accountmanager.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/connectingtocluster/index.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/connectingtocluster/mac.html",
"https://rc-docs.northeastern.edu/en/latest/gettingstarted/connectingtocluster/windows.html",
"https://rc-docs.northeastern.edu/en/latest/first_steps/index.html",
"https://rc-docs.northeastern.edu/en/latest/first_steps/passwordlessssh.html",
"https://rc-docs.northeastern.edu/en/latest/first_steps/shellenvironment.html",
"https://rc-docs.northeastern.edu/en/latest/first_steps/usingbash.html",
"https://rc-docs.northeastern.edu/en/latest/hardware/index.html",
"https://rc-docs.northeastern.edu/en/latest/hardware/hardware_overview.html",
"https://rc-docs.northeastern.edu/en/latest/hardware/partitions.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/index.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/introduction.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/accessingood.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/interactiveapps/index.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/interactiveapps/desktopood.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/interactiveapps/fileexplore.html",
"https://rc-docs.northeastern.edu/en/latest/using-ood/interactiveapps/jupyterlab.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/index.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/understandingqueuing.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/jobscheduling.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/jobscheduling.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/workingwithgpus.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/recurringjobs.html",
"https://rc-docs.northeastern.edu/en/latest/runningjobs/debuggingjobs.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/index.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/discovery_storage.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/transferringdata.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/globus.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/databackup.html",
"https://rc-docs.northeastern.edu/en/latest/datamanagement/securityandcompliance.html",
"https://rc-docs.northeastern.edu/en/latest/software/index.html",
"https://rc-docs.northeastern.edu/en/latest/software/systemwide/index.html",
"https://rc-docs.northeastern.edu/en/latest/software/systemwide/modules.html",
"https://rc-docs.northeastern.edu/en/latest/software/systemwide/mpi.html",
"https://rc-docs.northeastern.edu/en/latest/software/systemwide/r.html",
"https://rc-docs.northeastern.edu/en/latest/software/systemwide/matlab.html",
"https://rc-docs.northeastern.edu/en/latest/software/packagemanagers/index.html",
"https://rc-docs.northeastern.edu/en/latest/software/packagemanagers/conda.html",
"https://rc-docs.northeastern.edu/en/latest/software/packagemanagers/spack.html",
"https://rc-docs.northeastern.edu/en/latest/software/fromsource/index.html",
"https://rc-docs.northeastern.edu/en/latest/software/fromsource/makefile.html",
"https://rc-docs.northeastern.edu/en/latest/software/fromsource/cmake.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/index.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/introductiontoslurm.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmcommands.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmrunningjobs.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmmonitoringandmanaging.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmscripts.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmarray.html",
"https://rc-docs.northeastern.edu/en/latest/slurmguide/slurmbestpractices.html",
"https://rc-docs.northeastern.edu/en/latest/classroom/index.html",
"https://rc-docs.northeastern.edu/en/latest/classroom/class_use.html",
"https://rc-docs.northeastern.edu/en/latest/classroom/cps_ood.html",
"https://rc-docs.northeastern.edu/en/latest/classroom/classroomexamples.html",
"https://rc-docs.northeastern.edu/en/latest/best-practices/index.html",
"https://rc-docs.northeastern.edu/en/latest/best-practices/homequota.html",
"https://rc-docs.northeastern.edu/en/latest/best-practices/checkpointing.html",
"https://rc-docs.northeastern.edu/en/latest/best-practices/optimizingperformance.html",
"https://rc-docs.northeastern.edu/en/latest/best-practices/software.html",
"https://rc-docs.northeastern.edu/en/latest/tutorialsandtraining/index.html",
"https://rc-docs.northeastern.edu/en/latest/tutorialsandtraining/canvasandgithub.html",
"https://rc-docs.northeastern.edu/en/latest/faq.html",
"https://rc-docs.northeastern.edu/en/latest/glossary.html",
]
loader = WebBaseLoader(urls)
data = loader.load()


In [17]:
import tiktoken
encoding_name = tiktoken.get_encoding("cl100k_base")
#tiktoken package to count the total tokens in our corpus
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [18]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=25)
docs = text_splitter.split_documents(data)

'''
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 70,
    length_function = len,
    add_start_index = True,
)
docs = text_splitter.create_documents([data])

for idx, text in enumerate(docs):
    docs[idx].metadata['source'] = "RCDocs"
'''

'\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\n\ntext_splitter = RecursiveCharacterTextSplitter(\n    chunk_size = 700,\n    chunk_overlap  = 70,\n    length_function = len,\n    add_start_index = True,\n)\ndocs = text_splitter.create_documents([data])\n\nfor idx, text in enumerate(docs):\n    docs[idx].metadata[\'source\'] = "RCDocs"\n'

In [19]:
type(docs[0])

langchain.schema.document.Document

In [20]:
docs[0]

Document(page_content='\n\n\n\n\n\n\nResearch Computing - RC RTD\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\n\n\n\n\n\nMenu\n\n\n\n\n\n\n\nExpand\n\n\n\n\n\nLight mode\n\n\n\n\n\n\n\n\n\n\n\n\n\nDark mode\n\n\n\n\n\n\nAuto light/dark mode\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHide navigation sidebar\n\n\nHide table of contents sidebar\n\n\n\n\n\nToggle site navigation sidebar\n\n\n\n\nRC RTD\n\n\n\n\nToggle Light / Dark / Auto color theme\n\n\n\n\n\n\nToggle table of contents sidebar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nResearch ComputingToggle child pages in navigation\nWelcome\nServices We Provide\nGetting Help\nIntroduction to HPC and Slurm\nCase Studies and User Testimonials\n\n\n\n\nGetting StartedToggle child pages in navigation\nGetting Access\nAccount Manager\nConnecting To ClusterToggle child pages in navigation\nMac\nWindows\n\n\n\n\nFirst StepsToggle child pages in navigation\nPasswordless SSH\nShell Environment on the Cluster\nCluster via Command-Line\n\n\n\nUser Guides\n

In [21]:
#from langchain.vectorstores import Pinecone
#import pinecone
from langchain.embeddings import CohereEmbeddings


In [22]:
embeddings = CohereEmbeddings(model='embed-english-light-v2.0',cohere_api_key=os.environ.get("COHERE_API_KEY"))


In [23]:
#pinecone.init(
#	api_key='b360318b-4fc8-4580-bf6c-d88959179985',
#	environment='us-west1-gcp-free'
#)

In [24]:
#pinecone.delete_index("chatbot1")


In [25]:
#pinecone.create_index("chatbot1", dimension=1024)


In [26]:
#index = pinecone.Index('chatbot1')

#index_name = "chatbot1"

In [27]:
#docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [28]:
from langchain.vectorstores import Chroma


In [29]:
'''
#You can configure Chroma to save and load from your local machine. 
#Data will be persisted automatically and loaded on start (if it exists).

# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)



'''

'\n#You can configure Chroma to save and load from your local machine. \n#Data will be persisted automatically and loaded on start (if it exists).\n\nimport chromadb\n\nfrom chromadb.config import Settings\n\n\nclient = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", \n                                    persist_directory="db/"\n                                ))\n#DuckDB on the backend\n#stored in db folder\n'

In [30]:
db = Chroma.from_documents(docs, embeddings)


In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
#from langchain.vectorstores import Pinecone


In [32]:
# load index
#docsearch = Pinecone.from_existing_index(index_name, embeddings)


In [33]:
# initialize base retriever
#retriever = docsearch.as_retriever(search_kwargs={"k": 4})
#retriever = Chroma.as_retriever(search_kwargs={"k": 4})
#retriever = Chroma.as_retriever(
#    search_type="mmr",
#    search_kwargs={'k': 4, 'fetch_k': 50} )

from langchain.schema.vectorstore import VectorStoreRetriever
retriever = VectorStoreRetriever(vectorstore=db, search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 10},)

'''

VectorStoreRetriever

Return VectorStoreRetriever initialized from this VectorStore.

Args:
    search_type (Optional[str]): Defines the type of search that
        the Retriever should perform.
Can be "similarity" (default), "mmr", or
"similarity_score_threshold".
    search_kwargs (Optional[Dict]): Keyword arguments to pass to the
        search function. Can include things like:
            k: Amount of documents to return (Default: 4)
            score_threshold: Minimum relevance threshold
                for similarity_score_threshold
            fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
            lambda_mult: Diversity of results returned by MMR;
                1 for minimum diversity and 0 for maximum. (Default: 0.5)
            filter: Filter by document metadata

Returns:
    VectorStoreRetriever: Retriever class for VectorStore.

Examples:

# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
docsearch.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)

# Fetch more documents for the MMR algorithm to consider
# But only return the top 5
docsearch.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)

# Only retrieve documents that have a relevance score
# Above a certain threshold
docsearch.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)

# Only get the single most similar document from the dataset
docsearch.as_retriever(search_kwargs={'k': 1})

# Use a filter to only retrieve documents from a specific paper
docsearch.as_retriever(
    search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}}
)
'''

'\n\nVectorStoreRetriever\n\nReturn VectorStoreRetriever initialized from this VectorStore.\n\nArgs:\n    search_type (Optional[str]): Defines the type of search that\n        the Retriever should perform.\nCan be "similarity" (default), "mmr", or\n"similarity_score_threshold".\n    search_kwargs (Optional[Dict]): Keyword arguments to pass to the\n        search function. Can include things like:\n            k: Amount of documents to return (Default: 4)\n            score_threshold: Minimum relevance threshold\n                for similarity_score_threshold\n            fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)\n            lambda_mult: Diversity of results returned by MMR;\n                1 for minimum diversity and 0 for maximum. (Default: 0.5)\n            filter: Filter by document metadata\n\nReturns:\n    VectorStoreRetriever: Retriever class for VectorStore.\n\nExamples:\n\n# Retrieve more documents with higher diversity\n# Useful if your dataset has 

In [34]:
compressor = CohereRerank() #LLMChainExtractor,LLMChainFilter,EmbeddingsFilter
# will iterate over the initially returned documents and extract from each only the content that is relevant to the query.

In [35]:
# Set up cohere's reranker
''' instead of immediately returning retrieved documents as-is, 
you can compress them using the context of the given query, so that only the relevant information is returned. '''
reranker = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


In [36]:
from langchain.memory import ConversationTokenBufferMemory


In [37]:
from langchain.chat_models import ChatOpenAI


In [38]:
#from langchain.callbacks import ContextCallbackHandler
#from langchain.callbacks import FlyteCallbackHandler
from langchain.callbacks import StdOutCallbackHandler


In [39]:
#context_callback = ContextCallbackHandler(token="T1gM1n4RzGWLFSsJnQ5ziLUW")
#context_callback = FlyteCallbackHandler()
context_callback = StdOutCallbackHandler()

In [40]:
llm = ChatOpenAI(temperature=0.7, verbose=True, openai_api_key = os.environ.get("OPENAI_API_KEY"), streaming=True, callbacks=[context_callback])


In [41]:
'''
ConversationTokenBufferMemory keeps a buffer of recent interactions in memory,
and uses token length rather than number of interactions to determine when to flush interactions.
'''
memory = ConversationTokenBufferMemory(llm=llm,memory_key="chat_history", return_messages=True,input_key='question',max_token_limit=1000)

In [42]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain import PromptTemplate

CONDENSE_QUESTION_PROMPT = '''
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.
Generate a search query based on the conversation and the new question.

Chat History:
{chat_history}

Question:
{question}

Search query:
'''



PromptTemplates = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.
Generate a search query based on the conversation and the new question.

Chat History:
{chat_history}

Question:
{question}

Search query:"""
)


system_message_prompt = SystemMessagePromptTemplate(prompt=PromptTemplates)

chat_prompt_for_ques = ChatPromptTemplate.from_messages(
    [system_message_prompt])



In [43]:
from langchain.chains import LLMChain


In [44]:
question_generator = LLMChain(llm=llm, prompt=chat_prompt_for_ques, verbose=True)


In [45]:
Answer_Generator_Prompt= '''
<Instructions>
Important:
Answer with the facts listed in the list of sources below. If there isn't enough information below, say you don't know.
If asking a clarifying question to the user would help, ask the question.
ALWAYS return a "SOURCES" part in your answer, except for small-talk conversations.

Question: {question}
Sources:
---------------------
    {summaries}
---------------------

Chat History:
{chat_history}
'''

In [46]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chat_prompt = PromptTemplate(template=Answer_Generator_Prompt, input_variables=["question", "summaries","chat_history"])

answer_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True,prompt=chat_prompt)


In [47]:
from langchain.chains import ConversationalRetrievalChain


chain = ConversationalRetrievalChain(
            retriever=reranker,
            question_generator=question_generator,
            combine_docs_chain=answer_chain,
            verbose=True,
            memory=memory,
            rephrase_question=False
)


In [48]:
query = "What is the Scheduling Policies for HPC cluster?"
result = chain({"question": query})


print("Question from user : " , query ,"\n")
print("Reply from ChatBot : " , result['answer'])






> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

<Instructions>
Important:
Answer with the facts listed in the list of sources below. If there isn't enough information below, say you don't know.
If asking a clarifying question to the user would help, ask the question.
ALWAYS return a "SOURCES" part in your answer, except for small-talk conversations.

Question: What is the Scheduling Policies for HPC cluster?
Sources:
---------------------
    Content:  Job Scripts
Slurm Array Jobs and Dependencies
Slurm Best Practices


HPC for the ClassroomToggle child pages in navigation
Classroom HPC: FAQ
CPS Class Instructions


Best PracticesToggle child pages in navigation
Home Directory Storage Quota
Checkpointing Jobs
Optimizing Job Performance
Best SW Practices


Tutorials and TrainingToggle child pages in navigation
Canvas and GitHub


Frequently Asked Questions
Glossary

Contribu

In [53]:
query = "How do I check Job Status?"
result = chain({"question": query})


print("Question from user : " , query ,"\n")
print("Reply from ChatBot : " , result['answer'])






> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base.
Generate a search query based on the conversation and the new question.

Chat History:

Human: What is the Scheduling Policies for HPC cluster?
Assistant: The scheduling policies for an HPC (High-Performance Computing) cluster include:
1. FIFO (First-In-First-Out): Jobs are executed in the order they are submitted, which may result in long wait times for large jobs if smaller jobs are constantly being submitted.
2. Fair Share: This policy ensures that all users receive a fair share of cluster resources over time, even if some users have high resource usage.
3. Priority-Based: Jobs are assigned priorities based on user-defined criteria or system-wide rules. Higher-priority jobs are executed before lower-priority ones, allow

In [54]:
type(memory)

langchain.memory.token_buffer.ConversationTokenBufferMemory

In [55]:
memory

ConversationTokenBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What is the Scheduling Policies for HPC cluster?'), AIMessage(content='The scheduling policies for an HPC (High-Performance Computing) cluster include:\n1. FIFO (First-In-First-Out): Jobs are executed in the order they are submitted, which may result in long wait times for large jobs if smaller jobs are constantly being submitted.\n2. Fair Share: This policy ensures that all users receive a fair share of cluster resources over time, even if some users have high resource usage.\n3. Priority-Based: Jobs are assigned priorities based on user-defined criteria or system-wide rules. Higher-priority jobs are executed before lower-priority ones, allowing for resource allocation based on user requirements.\n\nJob priorities in an HPC cluster can be determined by factors such as user priority, resource requirements, and walltime limit.\n\nSources:\n- "Job Scheduling Policies and Priorities" from https://

In [52]:
'''
Conversation Memory Buffer & Input Details:
------------------------------------------
- Input Key: question
- Memory Key: chat_history
- Maximum Token Limit: 1000
- Return Messages: True

ChatOpenAI Configuration:
-------------------------
- Verbose: True
- Streaming: True
- OpenAI API Key: [REDACTED]
- OpenAI API Base: (No Value Provided)
- OpenAI Organization: (No Value Provided)
- OpenAI Proxy: (No Value Provided)
- Client: ChatCompletion
- Callbacks: FlyteCallbackHandler (Object Reference: 0x7832be39c9d0)

Conversation History:
---------------------
1. User: What is Globus at Northeastern?

   Assistant: Globus is a data management system that allows you to transfer and share files.
   Northeastern has a subscription to Globus, and you can set up a Globus account using your Northeastern credentials.
   You can also link your other accounts, whether they are personal or from another institution, with your Globus account.
   To use Globus, you will need to set up an account and install Globus Connect on your local computer.
   After completing these initial setup procedures, you can use the Globus web app to perform file transfers.

   Sources:
   - RCDocs (Using Globus)
   - RCDocs (Globus Account Set Up)
   - RCDocs (To use Globus)

2. User: How do i set up an account with Globus?

   Assistant: Globus is a data management system that allows you to transfer and share files.
   Northeastern has a subscription to Globus, and you can set up an account with Globus using your Northeastern credentials.
   You can also link your other accounts, whether they are personal or from another institution, with your Globus account.
   To set up an account with Globus, follow these steps:
   1. Go to the Globus website.
   2. Click on "Log In".
   3. Select "Northeastern University" from the options under "Use your existing organizational login" and click "Continue".
   4. Enter your Northeastern username and password.
   5. If you don't have a previous Globus account, click "Continue". If you have an existing account, click "Link to an existing account".
   6. Check the agreement checkbox and click "Continue".
   7. Click "Allow" to permit Globus to access your files.
   After setting up your account, you can access the Globus File Manager app.

   Sources:
   - RCDocs (Using Globus)
   - RCDocs (Globus Account Set Up)
'''


'\nConversation Memory Buffer & Input Details:\n------------------------------------------\n- Input Key: question\n- Memory Key: chat_history\n- Maximum Token Limit: 1000\n- Return Messages: True\n\nChatOpenAI Configuration:\n-------------------------\n- Verbose: True\n- Streaming: True\n- OpenAI API Key: [REDACTED]\n- OpenAI API Base: (No Value Provided)\n- OpenAI Organization: (No Value Provided)\n- OpenAI Proxy: (No Value Provided)\n- Client: ChatCompletion\n- Callbacks: FlyteCallbackHandler (Object Reference: 0x7832be39c9d0)\n\nConversation History:\n---------------------\n1. User: What is Globus at Northeastern?\n\n   Assistant: Globus is a data management system that allows you to transfer and share files.\n   Northeastern has a subscription to Globus, and you can set up a Globus account using your Northeastern credentials.\n   You can also link your other accounts, whether they are personal or from another institution, with your Globus account.\n   To use Globus, you will need t

In [58]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

2023-10-09 17:32:00.269435: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /device:GPU:0 with 13605 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


['/device:GPU:0']

2023-10-09 17:31:31.349324: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /device:GPU:0 with 13605 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5


['/device:GPU:0']